# TensorFlow Core

This guide gets you started programming in the low-level TensorFlow APIs (TensorFlow Core), showing you how to:
-  Manage your own TensorFlow program (a `tf.Graph`) and TensorFlow runtime (a `tf.Session`), instead of relying on Estimators to manage them.
-  Run TensorFlow operations, using a `tf.Session`.
-  Use high level components (datasets, layers, and feature_columns) in this low level environment.
-  Build your own training loop, instead of using the one provided by Estimators.

We recommend using the higher level APIs to build models when possible. Knowing TensorFlow Core is valuable for the following reasons:
-  Experimentation and debugging are both more straight forward when you can use low level TensorFlow operations directly.
-  It gives you a mental model of how things work internally when using the higher level APIs.

## Setup

Before using this guide, install TensorFlow.

To get the most out of this guide, you should know the following:
-  How to program in Python.
-  At least a little bit about arrays.
-  Ideally, something about machine learning.

Feel free to launch `python` and follow along with this walkthrough. Run the following lines to set up your Python environment:

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

## Tensor Values

The central unit of data in TensorFlow is the __tensor__. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's __rank__ is its number of dimensions, while its __shape__ is a tuple of integers specifying the array's length along each dimension. Here are some examples of tensor values:

```
 3. # a rank 0 tensor; a scalar with shape [],
 [1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
 [[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
 [[[1., 2., 3.]], [[4., 5., 6.]]] # a rank 3 tensor with shape [2, 1, 3]
```

TensorFlow uses numpy arrays to represent tensor __values__.

## TensorFlow Core Walkthrough

You might think of TensorFlow Core programs as consisting of two discrete sections:
1. Build the computational graph (a `tf.Graph`).
2. Running the computational graph (using a `tf.Session`).

## Graph

A __computational graph__ is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.
-  `tf.Operation` (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
-  `tf.Tensor`: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return `tf.Tensors`.

__Important: tf.Tensors__ do not have values, they are just handles to elements in the computation graph.

Let's build a simple computational graph. The most basic operation is a constant. The Python function that builds the operation takes a tensor value as input. The resulting operation takes no inputs. When run, it outputs the value that was passed to the constructor. We can create two floating point constants `a` and `b` as follows:

In [2]:
a = tf.constant(3.0, dtype = tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


Notice that printing the tensors does not output the values `3.0`, `4.0`, and `7.0` as you might expect. The above statements only build the computation graph. These `tf.Tensor` objects just represent the results of the operations that will be run.

Each operation in a graph is given a unique name. This name is independent of the names the objects are assigned to in Python. Tensors are named after the operation that produces them followed by an output index, as in `"add:0"` above.

## TensorBoard

TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

This will produce an `event` file in the current directory with a name in the following format:
```
 events.out.tfevents.{timestamp}.{hostname}
```
Now, in a terminal, launch TensorBoard with the following shell command:

```
 tensorboard --logdir .
```
Then open TensorBoard's graphs page in your browser, and you should see a graph similar to the following:

<img src="files/graphs.png" width="300">

For more about TensorBoard's graph visualization tools see TensorBoard: Graph Visualization.

## Session

To evaluate tensors, instantiate a `tf.Session` object, informally known as a __session__. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a `tf.Graph` is like a `.py` file, a `tf.Session` is like the `python` executable.

The following code creates a `tf.Session` object and then invokes its `run` method to evaluate the `total` tensor we created above:

In [4]:
sess = tf.Session()
print(sess.run(total))

7.0


When you request the output of a node with `Session.run` TensorFlow backtracks through the graphs and runs all the nodes that provide input to the requested output node. So this prints the expected value of 7.0.

You can pass multiple tensors to `tf.Session.run`. The `run` method transparently handles any combination of tuples or dictionaries, as in the following example:

In [5]:
print(sess.run({'ab': (a, b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


During a call to `tf.Session.run` any `tf.Tensor` only has a single value. For example, the following code calls `tf.random_uniform` to produce a `tf.Tensor` that generates a random 3-element vector (with values in `[0, 1)`):

In [6]:
vec = tf.random_uniform(shape = (3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.4205289  0.20415878 0.34228337]
[0.3524592  0.14103198 0.97781515]
(array([1.7280518, 1.8744731, 1.4172102], dtype=float32), array([2.7280517, 2.874473 , 2.41721  ], dtype=float32))


The result shows a different random value on each call to `run`, but a consistent value during a single `run` (`out1` and `out2` receive the same random input).

Some TensorFlow 